# Importação dos módulos e Leitura da base de dados

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install unidecode

In [ ]:
import pandas as pd
import nltk
from nltk import tokenize
from sklearn.feature_extraction.text import CountVectorizer
from string import punctuation
import unidecode
from gensim.models import KeyedVectors
import string
import nltk
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [ ]:
nltk.download('rslp')

In [ ]:
nltk.download("punkt")

In [ ]:
!unzip "/content/drive/MyDrive/Mineração de Texto e Processamento de Linguagem Natural/Material de Apoio/criticas-imdb.zip"

In [ ]:
!unzip "/content/drive/My Drive/Mineração de Texto e Processamento de Linguagem Natural/Material de Apoio/cbow_s300.zip"

In [ ]:
!unzip "/content/drive/My Drive/Mineração de Texto e Processamento de Linguagem Natural/Material de Apoio/skip_s300.zip"

In [ ]:
criticas = pd.read_csv("criticas-imdb.csv")

# Limpeza dos textos de crítica

### Eliminando acentuação

In [ ]:
tokenizadorPontuacaoEspacos = tokenize.WordPunctTokenizer()

In [ ]:
textos_criticas_sem_acentos = [unidecode.unidecode(texto) for texto in list(criticas.texto)]
textos_criticas_sem_acentos

In [ ]:
textos_processados = list()
for texto in textos_criticas_sem_acentos:
  tokens_texto = tokenizadorPontuacaoEspacos.tokenize(texto)
  tokens_texto_limpo = list()
  for token in tokens_texto:
    tokens_texto_limpo.append(token)
  textos_processados.append(' '.join(tokens_texto_limpo))

### Eliminando as palavras irrelevantes e a pontuação dos textos

In [ ]:
nltk.download('stopwords')
palavras_irrelevantes = nltk.corpus.stopwords.words('portuguese')

In [ ]:
# Função para colocar as pontuações em uma lista
pontuacoes = list()
for pontuacao in punctuation:
  pontuacoes.append(pontuacao)

In [ ]:
# Pontuações e palavras irrelevantes em uma lista só
pontuacoes_e_palavras_irrelevantes = pontuacoes + palavras_irrelevantes

In [ ]:
# Função para eliminar as palavras irrelevantes e a pontuação
textos_processados_2 = list()
for texto in textos_processados:
  tokens_texto = tokenizadorPontuacaoEspacos.tokenize(texto)
  tokens_texto_limpo = list()
  for token in tokens_texto:
    if token not in pontuacoes_e_palavras_irrelevantes:
      tokens_texto_limpo.append(token)
  textos_processados_2.append(' '.join(tokens_texto_limpo))

### Deixando todo o texto com letras minúsculas

In [ ]:
textos_processados_3 = list()
for texto in textos_processados_2:
  texto = texto.lower()
  tokens_texto = tokenizadorPontuacaoEspacos.tokenize(texto)
  tokens_texto_limpo = list()
  for token in tokens_texto:
    # if token not in pontuacoes_e_palavras_irrelevantes_sem_acentos:
    tokens_texto_limpo.append(token)
  textos_processados_3.append(' '.join(tokens_texto_limpo))

### Stemização dos textos

In [ ]:
stemizador = nltk.RSLPStemmer()

In [ ]:
textos_processados_4 = list()
for texto in textos_processados_3:
  tokens_texto = tokenizadorPontuacaoEspacos.tokenize(texto)
  tokens_stemizados = [stemizador.stem(token) for token in tokens_texto]
  textos_processados_4.append(' '.join(tokens_stemizados))

## Colocando a lista dos textos tratados na base de dados

In [ ]:
criticas["textos_tratados"]=textos_processados_4

# Divisão da base de treino e teste

In [ ]:
criticas_treino, criticas_teste = train_test_split(criticas, test_size=0.2, random_state = 10)

# Funções que auxiliarão a classificação dos textos

In [ ]:
# Função para tokenizar os textos
def tratarTokenizar(texto):
  texto=texto.lower()
  tokens_validos = []

  for token in nltk.word_tokenize(texto):
    if token not in string.punctuation:
      tokens_validos.append(token)

  return tokens_validos

In [ ]:
# Função para somar valores para cada token do texto de crítica de acordo com o modelo
def somaVetores(tokens, modelo):
  vetorSomado = np.zeros(300)
  for token in tokens:
    try:
      vetorSomado += modelo.get_vector(token)
    except:
      if token.isnumeric() and len(token) <= 15:
        token = "0"*len(token)
        vetorSomado += modelo.get_vector(token)
      elif token.isnumeric() and len(token) > 15:
        token = "0"*15
        vetorSomado += modelo.get_vector(token)
      else:
        vetorSomado += modelo.get_vector("unknown")
        
  return vetorSomado

In [ ]:
# Função para construir a base de dados que será treinada e testada de acordo com o modelo
def vetoresTextos(textos, modelo):
  x = len(textos)
  y = 300
  vetores = np.zeros((x,y))

  for itexto in range(x):
    tokens = tratarTokenizar(textos.iloc[itexto])
    vetores[itexto] = somaVetores(tokens, modelo)

  return vetores

# Classificando com modelo Word2Vec (CBOW) - Textos tratados

In [ ]:
# Importando o arquivo txt com o modelo cbow
modelo_cbow = KeyedVectors.load_word2vec_format("cbow_s300.txt")

### Cálculo dos vetores de treino e de teste

In [ ]:
vetores_treino = vetoresTextos(criticas_treino.textos_tratados, modelo_cbow)
vetores_teste = vetoresTextos(criticas_teste.textos_tratados, modelo_cbow)

### Geração do Modelo e avaliação da classificação

In [ ]:
classificador_cbow = LogisticRegression(max_iter=400)

In [ ]:
classificador_cbow.fit(vetores_treino, criticas_treino.sentimento)

In [ ]:
classificador_cbow.score(vetores_teste, criticas_teste.sentimento)

In [ ]:
sentimentos_previstos = classificador_cbow.predict(vetores_teste)
relatorio = classification_report(criticas_teste.sentimento, sentimentos_previstos)
print(relatorio)

              precision    recall  f1-score   support

    negativo       0.77      0.75      0.76      4991
    positivo       0.75      0.77      0.76      4901

    accuracy                           0.76      9892
   macro avg       0.76      0.76      0.76      9892
weighted avg       0.76      0.76      0.76      9892



# Classificando com modelo Word2Vec (CBOW) - Textos não tratados

### Cálculo dos vetores de treino e de teste

In [ ]:
# Cálculo dos vetores de treino e de teste
vetores_treino2 = vetoresTextos(criticas_treino.texto, modelo_cbow)
vetores_teste2 = vetoresTextos(criticas_teste.texto, modelo_cbow)

### Geração do Modelo e avaliação da classificação

In [ ]:
classificador_cbow2 = LogisticRegression(max_iter=400)

In [ ]:
classificador_cbow2.fit(vetores_treino2, criticas_treino.sentimento)

In [ ]:
classificador_cbow2.score(vetores_teste2, criticas_teste.sentimento)

In [ ]:
sentimentos_previstos2 = classificador_cbow2.predict(vetores_teste)
relatorio2 = classification_report(criticas_teste.sentimento, sentimentos_previstos2)
print(relatorio2)

              precision    recall  f1-score   support

    negativo       0.81      0.81      0.81      4991
    positivo       0.80      0.80      0.80      4901

    accuracy                           0.80      9892
   macro avg       0.80      0.80      0.80      9892
weighted avg       0.80      0.80      0.80      9892



# Classificando com modelo Word2Vec (SkipGram) - Textos tratados

In [ ]:
# Importando o arquivo txt com o modelo SkipGram
modelo_skipgram = KeyedVectors.load_word2vec_format("skip_s300.txt")

### Cálculo dos vetores de treino e de teste

In [ ]:
vetores_treino_skipgram = vetoresTextos(criticas_treino.textos_tratados, modelo_skipgram)
vetores_teste_skipgram = vetoresTextos(criticas_teste.textos_tratados, modelo_skipgram)

### Geração do Modelo e avaliação da classificação

In [ ]:
classificador_skipgram = LogisticRegression(max_iter=400)

In [ ]:
classificador_skipgram.fit(vetores_treino_skipgram, criticas_treino.sentimento)

In [ ]:
sentimentos_previstos_skipgram = classificador_skipgram.predict(vetores_teste_skipgram)
relatorio_skipgram = classification_report(criticas_teste.sentimento, sentimentos_previstos_skipgram)
print(relatorio_skipgram)

              precision    recall  f1-score   support

    negativo       0.77      0.75      0.76      4991
    positivo       0.75      0.78      0.77      4901

    accuracy                           0.76      9892
   macro avg       0.76      0.76      0.76      9892
weighted avg       0.76      0.76      0.76      9892



# Classificando com modelo Word2Vec (SkipGram) - Textos não tratados

### Cálculo dos vetores de treino e de teste

In [ ]:
vetores_treino_skipgram2 = vetoresTextos(criticas_treino.texto, modelo_skipgram)
vetores_teste_skipgram2 = vetoresTextos(criticas_teste.texto, modelo_skipgram)

### Geração do Modelo e avaliação da classificação

In [ ]:
classificador_skipgram2 = LogisticRegression(max_iter=400)

In [ ]:
classificador_skipgram2.fit(vetores_treino_skipgram2, criticas_treino.sentimento)

In [ ]:
sentimentos_previstos_skipgram2 = classificador_skipgram2.predict(vetores_teste_skipgram2)
relatorio_skipgram2 = classification_report(criticas_teste.sentimento, sentimentos_previstos_skipgram2)
print(relatorio_skipgram2)

              precision    recall  f1-score   support

    negativo       0.81      0.81      0.81      4991
    positivo       0.81      0.81      0.81      4901

    accuracy                           0.81      9892
   macro avg       0.81      0.81      0.81      9892
weighted avg       0.81      0.81      0.81      9892



# Avaliação dos resultados

Observando os resultados gerados, pode-se perceber que os classificadores do modelo CBOW e SkipGram obtiveram resultados semelhantes. 
 


Para os textos não-tratados sobre as críticas, apenas a precisão e a revogação para as críticas que foram rotuladas como positivas tiveram ligeira melhora no modelo SkipGram, saindo de 0,8 no modelo CBOW e indo para 0,81.

Para os textos tratados, os resultados de performance em relação aos modelos foram praticamente os mesmos, variando entre 0,75 e 0,78. Pode-se perceber também que o tratamento dos textos (eliminação de palavras irrelevantes, pontuação, acentuação e stemização das palavras derivadas) piorou a performance dos dois modelos.

Comparando esses resultados com os que foram obtidos com o uso da técnica bag-of-words na Unidade 2 pode-se perceber que ocorreu um resultado inverso. 

Na técnica bag-of-words, o classificador teve uma performance de 0.65 de acurácia com os textos não-tratados. Essa performance chegou a 0.82 com a eliminação de palavras irrelevantes, pontuações e acentuações além da stemização das palavras derivadas e, por fim, chegou a 0.88 com o uso da técnica TF-IDF e dos bigramas de palavras. 

Já nesta técnica que usou os modelos CBOW e SkipGram, o tratamento dos textos fez com que as performances dos classificadores piorassem.